# WEB SCRAPING

<p align = "justify">
    This project is about web scraping using this two python library: Selenium and Openyxl.
    Follow these steps bello.
</p>

### STEP 1: requirements

In [1]:
!pip3 install selenium openpyxl plotly_express
!pip3 install -U kaleido

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
# --------------------------------------------- Web Scraping ---------------------------------------------

from selenium import webdriver                  # Simula o uso do navegador
from selenium.webdriver.common.by import By     # Permite encontrar informações dentro de sites
import openpyxl                                 # Permite criar e manipular planilhas

# ---------------------------------------- Análise de Dados (Gráfico) ---------------------------------------

import pandas as pd             # Abrir e manipular arquivos em excel
import plotly.express as px     # Gerar gráficos interativos
import plotly                   # Usado para baixar o gráfico

### STEP 2: Creating the database (excel)

In [3]:
# ----------------------------------------------- ABRINDO A SITE ----------------------------------------------- #

# Abrindo o navegador Chrome e o site
    # Inicializando o navegador pelo webdriver.
    # Estou utilizando o navegador Chrome (por isso webdriver.Chrome())
    # Entrarei na seguinte loja: 

driver = webdriver.Chrome()
driver.get('https://www.kabum.com.br/computadores/monitores/monitor-gamer')

# ------------------------------------------- COLETANDO DADOS DA PÁGINA ------------------------------------------- #

# Informações sobre a coleta dos dados
    # xpath é uma forma de encontrar elementos dentro de um site
    # xpath strucutre = //tag[@attribute='value']
    # Title xpath = //span[@class='sc-d79c9c3f-0 nlmfp sc-93fa31de-16 bBOYrL nameCard']
    # Price xpath = //span[@class='sc-6889e656-2 bYcXfg priceCard']

# Coletando os dados utilizando o xpath
titles = driver.find_elements(By.XPATH,"//span[@class='sc-d79c9c3f-0 nlmfp sc-93fa31de-16 bBOYrL nameCard']")
promotional_prices = driver.find_elements(By.XPATH,"//span[@class='sc-6889e656-2 bYcXfg priceCard']")

# ------------------------------------------- INSERINDO DADOS NA PLANILHA ------------------------------------------- #

# Criando a planilha
planilha = openpyxl.Workbook()

# Criando a página 'produtos'
planilha.create_sheet('produtos')

# Selecionando a página 'produtos' e inserindo os rótulos (cabeçalhos)
sheet_produtos = planilha['produtos']
sheet_produtos['A1'].value = 'PRODUTO'
sheet_produtos['B1'].value = 'PREÇO PROMOCIONAL'

# Inserindo os dados na planilha (somente tuplas de nome, preço com valors diferentes de null para manter consistência)
for title, price in zip(titles, promotional_prices):
    sheet_produtos.append([title.text, price.text])

# Salvando a planilha
planilha.save('database.xlsx')

# Fechando o navegador
driver.quit()



### STEP 3: Formatting the data

In [4]:
# ----------------------------------------------- ABRINDO A PLANILHA ----------------------------------------------- #
database = openpyxl.load_workbook('database.xlsx')

# Acessando a página 'produtos'
sheet_produtos = database['produtos']

# ----------------------------------------------- FORMATANDO OS DADOS (PREÇO) ----------------------------------------------- #
# Excluindo excesso de informação nos preços
for cell in sheet_produtos['B']:
    cell.value = cell.value.replace('R$ ', '')  # Excluindo o 'R$ ' pois atrapalha a conversão para float
    cell.value = cell.value.replace('.', '')    # Excluindo o '.' pois também atrapalha a conversão para float
    cell.value = cell.value.replace(',', '.')   # Trocando a vírgula por ponto para evitar problemas com string e float

# Convertendo os preços para float (com exceção do cabeçalho)
for cell in sheet_produtos['B'][1:]:
  cell.value = float(cell.value)

# Salvando a planilha
database.save('database.xlsx')


### STEP 4: Ploting the line chart for analysis

In [7]:
# ----------------------------------------------- ABRINDO A PLANILHA ----------------------------------------------- #

# Abrindo a planilha
database = openpyxl.load_workbook('database.xlsx')

# Acessando a página 'produtos'
sheet_produtos = database['produtos']

# ---------------------------------------- ORDENANDO OS DADOS PARA O GRÁFICO ---------------------------------------- #

# Criando uma lista para armazenar os produtos (nome, preço)
produtos = []

# Inserindo os produtos na lista (OBS.: zip() é uma função que permite iterar sobre duas listas ao mesmo tempo)
for title, price in zip(sheet_produtos['A'][1:], sheet_produtos['B'][1:]):
  produtos.append([title.value, price.value])

# Ordenando a lista de produtos pelo preço (crescente)
produtos.sort(key=lambda produto: produto[1]) # key = lambda é um parâmetro que permite ordenar a lista por um valor específico, nesse caso o preço
 
# exibindo a lista de produtos com quebra de linha
#for produto in produtos:
# print(produto[0], produto[1], '\n')

# -------------------------------------------------- GERANDO O GRÁFICO -------------------------------------------------- #

# Criando um dataframe com os dados da lista de produtos
df = pd.DataFrame(produtos, columns=['Produto', 'Preço Promocional'])

# Gerando o gráfico de linhas
fig = px.line(
    df, 
    x = df.index,
    y = 'Preço Promocional',
    labels={'x': 'PRODUTO', 'Preço Promocional': 'PREÇO'},
    hover_data=['Produto'],
    markers=True,
    #color = 'Produto',
    title = 'BENCHMARK DE PREÇOS DE MONITORES GAMER - KABUM',
)

fig.show()
